In [1]:
import numpy as np
import pandas as pd
import load_file as lf

In [2]:
sigmoid = lambda x: 1 / (1 + np.exp(-x))

In [6]:
# TODO: Função update - alterar o valor dos x e passar adiante (u e g)
class Neuron:
  def __init__(self, x=None, w=None, function=sigmoid):
    self.x = x
    self.u = None
    self.pd = None
    self.function = function
    
    if type(self.x) is float or type(self.x) is int:
      # Bias ou entrada
      self.w = None
      self.g = self.x
    elif type(self.x) is list:
      # Normal
      self.w = w if type(w) is np.ndarray else np.random.random(len(self.x))
      self.u = self.x @ self.w
      self.g = self.function(self.u)
    else:
      # Não-inicializado
      self.w = w if w is None or type(w) is np.ndarray else np.random.random(w)
      self.g = None
      
  def update(self, new_x):
      self.x = new_x
      if not self.is_bias():
        self.u = self.x @ self.w
        self.g = self.function(self.u)
      else:
        self.g = self.x
  
  is_bias = lambda self: True if self.w is None else False
  __repr__ = lambda self: '{} @ {} -> {}'.format(self.x, self.w, self.g)

In [11]:
class MLP:
  def __init__(self, topology):
    self.topology = topology
    
    self.init_neurons()
#     self.inputs = [.05, .1]
#     self.desired = [.01, .99]
#     self.learning_rate = 0.5
#     self.layers = [[Neuron(.05), Neuron(.1), Neuron(1)]]
#     self.layers += [[Neuron([e.g for e in self.layers[0]], np.array([.15, .2, .35])), 
#                      Neuron([e.g for e in self.layers[0]], np.array([.25, .3, .35])), 
#                      Neuron(1)]]
#     self.layers += [[Neuron([e.g for e in self.layers[1]], np.array([.4, .45, .6])), 
#                      Neuron([e.g for e in self.layers[1]], np.array([.5, .55, .6]))]]
    
    
  def init_neurons(self):
    # Camada de entrada
    self.layers = [[Neuron() for i in range(self.topology[0])]]
    self.layers[0] += [Neuron(1)] # Bias 
    
    # Camadas ocultas e de saída
    for i in range(1, len(self.topology)):
      self.layers += [[Neuron(w=len(self.layers[i - 1])) for j in range(self.topology[i])]]
      if i != len(self.topology) - 1: self.layers[i] += [Neuron(1)] # Bias

  def update_neurons(self):
    
    # Camada de entrada
    [self.layers[0][i].update(self.inputs[i]) for i in range(len(self.inputs))]
    
    # Camadas ocultas e de saída
    for i in range(1, len(self.layers)):
      [n.update([e.g for e in self.layers[i - 1]]) for n in self.layers[i] if not n.is_bias()]
      
    print(self.layers, '\n')
#       print('Camada', i, '\n', self.layers[i])
        
  def update_weights(self):
    # Camada de saída
    for i in range(len(self.layers[-1])):
      # Derivada parcial
      neuron = self.layers[-1][i]
      neuron.pd = - (self.desired[i] - neuron.g) * neuron.g * (1 - neuron.g)
      # Pesos
      for j in range(len(neuron.w)):
        neuron.w[j] -= self.learning_rate * neuron.pd * self.layers[-2][j].g
          
    # Camadas ocultas
    for i in range(len(self.layers) - 2, 0, -1):
      for j in range(len(self.layers[i])):
        # Derivada parcial
        neuron = self.layers[i][j]
        descendents = sum([n.pd * n.w[j] for n in self.layers[i + 1] if not n.is_bias()])
        neuron.pd = descendents * neuron.g * (1 - neuron.g)
        # Pesos
        for k in range(len(neuron.w)):
          neuron.w[k] -= self.learning_rate * neuron.pd * neuron.x[k]
  
  def update_errors(self):
    out_layer = self.layers[-1]
    out_length = len(out_layer)
    
    self.errors = [(out_layer[i].g - self.desired[i]) ** 2 for i in range(out_length)]
    self.total_error = sum(self.errors) / 2
    
  def train(self, test_data, learning_rate=.1, precision=10**-6):
    for index, sample in test_data.iterrows():
      self.inputs = sample[:self.topology[0]].tolist()
      self.desired = sample[self.topology[0]:].tolist()
      
      self.update_neurons()
  
  def __repr__(self):
    result = 'Camada 0 - Entrada\n{}\n\n'.format(self.layers[0])
    for i in range(1, len(self.layers) - 1): result += 'Camada {} - Escondida \n{}\n\n'.format(i, self.layers[i])
    result += 'Camada {} - Saída\n{}\n\n'.format(len(self.layers) - 1, self.layers[-1])
    
    return result

In [40]:
df = pd.read_csv('../../datasets/xor.csv')
df = df.iloc[:1]

In [43]:
a = MLP([2, 3, 2, 2])
a.train(df)

[[0 @ None -> 0, 0 @ None -> 0, 1 @ None -> 1], [[0, 0, 1] @ [0.20363798 0.87531077 0.48961717] -> 0.6200162433481852, [0, 0, 1] @ [0.67462569 0.72748948 0.70954966] -> 0.6703016426004095, [0, 0, 1] @ [0.08450711 0.39602625 0.76317477] -> 0.6820426138210194, 1 @ None -> 1], [[0.6200162433481852, 0.6703016426004095, 0.6820426138210194, 1] @ [0.941715   0.17400685 0.11072998 0.12342108] -> 0.7108383793569891, [0.6200162433481852, 0.6703016426004095, 0.6820426138210194, 1] @ [0.91412153 0.12339591 0.34822553 0.65384699] -> 0.8235859794832703, 1 @ None -> 1], [[0.7108383793569891, 0.8235859794832703, 1] @ [0.10919312 0.55298947 0.62048998] -> 0.76015746824144, [0.7108383793569891, 0.8235859794832703, 1] @ [0.33673835 0.13619378 0.45871741] -> 0.6921630400959163]] 

